<div style="border: 1px solid black">
<b><center><font size="3">Data Acquisition</font></center></b>

<b><center><font size="2">Web Scraping Project</font></center></b>
</div>

**Notebook Developed by**: Rene Jerez </b>

**Notebook Co-writer by**: ChatGPT </b>

**Notebook Based on**: [Ricardo Campos](https://www.di.ubi.pt/~rcampos)<br>



<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span>
<a href="#Informação de uma página web estática 1" data-toc-modified-id="Informação de uma página web estática 1-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Informação de uma página web estática</a></span></span></li>
<li><span>
<a href="#Informação de uma página web dinâmica 2" data-toc-modified-id="Informação de uma página web dinâmica 2-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Informação de uma página web dinâmica</a></span>


### 1. Informação de uma página web estática

Obter melhores preços de tickets aereos da Companhia RyanAir com origem no Aeroporto do Porto

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urlencode
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [67]:
########################################
# Make a URL variable
########################################

# Base URL
base_url = 'https://www.ryanair.com/gb/en/cheap-flights-beta'

# Query parameters
payload = {
    'originIata': 'OPO',
    'destinationIata': 'ANY',
    'isReturn': 'true',
    'isMacDestination': 'false',
    'promoCode': '',
    'adults': '1',
    'teens': '0',
    'children': '0',
    'infants': '0',
    'dateOut': '2024-04-01',
    'dateIn': '2024-04-30',
    'daysTrip': '3',
    'dayOfWeek': 'FRIDAY',
    'isExactDate': 'false',
    'outboundFromHour': '00:00',
    'outboundToHour': '23:59',
    'inboundFromHour': '00:00',
    'inboundToHour': '23:59',
    'priceValueTo': '',
    'currency': 'EUR',
    'isFlexibleDay': 'false'
}

# Encode parameters
query_string = urlencode(payload)

# Full URL
ryanair_url = f"{base_url}?{query_string}"
#ryanair_url = 'https://www.ryanair.com/gb/en/cheap-flights-beta?originIata=OPO&destinationIata=ANY&isReturn=false&isMacDestination=false&promoCode=&adults=1&teens=0&children=0&infants=0&dateOut=2024-04-01&dateIn=2024-04-30&daysTrip=3&dayOfWeek=FRIDAY&isExactDate=false&outboundFromHour=00:00&outboundToHour=23:59&priceValueTo=&currency=EUR&isFlexibleDay=false'


In [56]:
########################################
# Start a webDriver
########################################
# Set up the Selenium WebDriver with Chrome
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run in background without opening a browser window
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

In [57]:

########################################
# Extract page content
########################################
driver.get(ryanair_url)
page_content = driver.page_source

In [58]:
########################################
# Save the content
########################################
with open('page.html', 'w', encoding='utf-8') as f:
    f.write(page_content)

driver.quit()

In [ ]:
########################################
# Find destinations in the full content
########################################

# Load HTML file
with open('page.html', 'r', encoding='utf-8') as file:
    html_content = file.read()

# Parse with Soup
soup = BeautifulSoup(html_content, 'html.parser')

# Find destinations classes
elements = soup.find_all(class_='country-card__content')
all_elements_html = ""


# Check if any elements were found
if elements:
    for element in elements:
        # Convert to a string
        element_html = str(element)

        # Append all elements
        all_elements_html += element_html + "\n\n"  # Add some spacing for readability
        
else:
    print("No destinations classes found.")


# Save file 'destinations.html'
if all_elements_html:
    with open('destinations.html', 'w', encoding='utf-8') as f:
        f.write(all_elements_html)
    print("File 'destinations.html' has been created with the extracted content.")
else:
    print("No content to write to 'destinations.html'.")

In [39]:
########################################
# Convert from HTML to Dataframe
########################################

# Load HTML file
with open('destinations.html', 'r', encoding='utf-8') as file:
    html_content = file.read()

# Parse HTML with Soup
soup = BeautifulSoup(html_content, 'html.parser')

# Info we need
countries = []
dates = []
prices = []

# Find elements 'country-card__content'
elements = soup.find_all(class_='country-card__content')

for element in elements:
    # Country
    country = element.find(class_='country-card__country').text.strip()
    countries.append(country)
    
    # Date
    date_info = element.find(class_='b2').text.strip()
    dates.append(date_info)
    
    # Price
    price = element.find(class_='country-card__price').text.strip()
    prices.append(price)

# Final df
df = pd.DataFrame({
    'Country': countries,
    'Date Info': dates,
    'Price': prices
})

In [40]:
df

,Country,Date Info,Price
0,Portugal 3 days in April,3 days in April,€90.79
1,Morocco 3 days in April,3 days in April,€101.28
2,Spain 3 days in April,3 days in April,€118.81
3,France 3 days in April,3 days in April,€124.08
4,United Kingdom 3 days in April,3 days in April,€143.23
5,Hungary 3 days in April,3 days in April,€191.62
6,Belgium 3 days in April,3 days in April,€198.55


### 2. Informação de uma página web Dinâmica

Obter Contratos Publicos recentes de origem da Covilhã

In [ ]:
name = 'get_contracts'
allowed_domains = ['base.gov.pt']
base_url = 'http://www.base.gov.pt/base2/rest/contratos'
ncontracts = 867164
step = 100

In [10]:
#base_url = 'http://www.base.gov.pt/base2/rest/contratos'
base_url = 'https://www.base.gov.pt/Base4/pt/pesquisa/?type=contratos&texto=&tipo=0&tipocontrato=0&cpv=&aqinfo=&adjudicante=&adjudicataria=&sel_price=price_c1&desdeprecocontrato=&ateprecocontrato=&desdeprecoefectivo=&ateprecoefectivo=&desdeprazoexecucao=&ateprazoexecucao=&sel_date=date_c1&desdedatacontrato=2024-03-01&atedatacontrato=2024-03-31&desdedatapublicacao=&atedatapublicacao=&desdedatafecho=&atedatafecho=&pais=187&distrito=6&concelho=68'
headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0 "}
#payload = { 'SomeParam': 'x', 'SomeOtherParam': 'y', 'AnotherOne': 'z'}

r = requests.get(base_url, headers = headers) #params=payload)



In [11]:
soup = BeautifulSoup(r.content, 'html.parser')
soup

<!DOCTYPE html>

<html lang="pt">
<head>
<title>Pesquisa</title>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="Pesquisa de contratos, anúncios, entidades, modificações contratuais ou bens móveis" name="description"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<link href="/Base4/Images/favicon.ico" rel="shortcut icon"/>
<link href="/Base4/Css/Bootstrap/bootstrap.min.css" rel="stylesheet"/>
<link href="/Base4/Css/FontAwesome/fontawesome.min.css" rel="stylesheet"/>
<link href="/Base4/Css/jQueryUI/jquery-ui.min.css" rel="stylesheet"/>
<link href="/Base4/Css/Custom/styles.min.css" rel="stylesheet"/>
</head>
<body>
<a class="sr-only sr-only-focusable" href="#main_content">Saltar para o conteúdo principal da página</a>
<div class="b-header-screen">
<div class="b-head-row-brand">
<div class="container-fluid b-page-width">
<div class="d-flex justify-content-between align-items-stretch b-header-block">
<div class="d-flex align-item

In [34]:
url_covilha = "https://www.base.gov.pt/Base4/pt/pesquisa/?type=contratos&texto=&tipo=0&tipocontrato=0&cpv=&aqinfo=&adjudicante=&adjudicataria=&sel_price=price_c1&desdeprecocontrato=&ateprecocontrato=&desdeprecoefectivo=&ateprecoefectivo=&desdeprazoexecucao=&ateprazoexecucao=&sel_date=date_c1&desdedatacontrato=2024-03-01&atedatacontrato=2024-03-31&desdedatapublicacao=&atedatapublicacao=&desdedatafecho=&atedatafecho=&pais=187&distrito=6&concelho=68"

In [27]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

# Initialize WebDriver with webdriver_manager
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Open the target URL
driver.get(url_covilha)

# Wait for the page to load completely
time.sleep(60)  # Adjust the sleep time as necessary

# Print the header or class name
print("Extracting content from table class='table table-striped'")

# Locate the table
table = driver.find_element(By.CLASS_NAME, 'table-striped')

# Extract and print the data from each row
for row in table.find_elements(By.TAG_NAME, "tr"):
    print(row.text)

# Close the browser
driver.quit()


Extracting content from table class='table table-striped'
Objeto do contrato Tipo de procedimento Adjudicante Adjudicatário Preço contratual Publicação
Consulta Prévia nº 02/SASUBI/2024 - Aquisição de Pão para os Serviços de Ação Social da Universidade da Beira Interior Consulta Prévia Serviços de Acção Social da Universidade da Beira Interior PANIFICADORA CENTRAL DO TORTOSENDO LDA 22.977,80 € 02-04-2024
Ajuste Direto nº 02/SASUBI/2024 - Aquisição de Bebidas Não Alcoólicas para os Serviços de Ação Social da Universidade da Beira Interior Ajuste Direto Regime Geral Serviços de Acção Social da Universidade da Beira Interior CONTACTO - REPRESENTAÇÕES E COMÉRCIO DE BEBIDAS, LDA 8.986,44 € 02-04-2024
Produtos de higiene Ajuste Direto Regime Geral Centro de Formação Profissional da Indústria Têxtil, Vestuário, Confecção e Lanifícios (Modatex) MUNDISAN - PAPEIS E PRODUTOS DE HIGIENE, LDA 3.588,26 € 02-04-2024
Aquisição de Serviços de Reparação de Telhado do Ovil Ajuste Direto Regime Geral Esc

In [36]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

# Initialize WebDriver with webdriver_manager
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Open the target URL
driver.get(url_covilha)

# Wait for the page to load completely
time.sleep(60)  # Adjust the sleep time as necessary

# Print the header or class name
print("Extracting content from table class='table table-striped'")

# Locate the table
table = driver.find_element(By.CLASS_NAME, 'table-striped')

# Assuming headers are in the first row
headers = [th.text for th in table.find_elements(By.TAG_NAME, "th")]

overall_dict = {}
for index, row in enumerate(table.find_elements(By.TAG_NAME, "tr")[1:], start=1):
    cells = row.find_elements(By.TAG_NAME, "td")
    row_dict = {headers[i]: cell.text for i, cell in enumerate(cells)}
    overall_dict[f"row_{index}"] = row_dict

# Close the browser
driver.quit()


Extracting content from table class='table table-striped'


In [37]:
df_rows = pd.DataFrame.from_dict(overall_dict, orient='index')

In [40]:
df_rows.head()

,Objeto do contrato,Tipo de procedimento,Adjudicante,Adjudicatário,Preço contratual,Publicação,
row_1,Consulta Prévia nº 02/SASUBI/2024 - Aquisição ...,Consulta Prévia,Serviços de Acção Social da Universidade da Be...,PANIFICADORA CENTRAL DO TORTOSENDO LDA,"22.977,80 €",02-04-2024,
row_2,Ajuste Direto nº 02/SASUBI/2024 - Aquisição de...,Ajuste Direto Regime Geral,Serviços de Acção Social da Universidade da Be...,CONTACTO - REPRESENTAÇÕES E COMÉRCIO DE BEBIDA...,"8.986,44 €",02-04-2024,
row_3,Produtos de higiene,Ajuste Direto Regime Geral,Centro de Formação Profissional da Indústria T...,"MUNDISAN - PAPEIS E PRODUTOS DE HIGIENE, LDA","3.588,26 €",02-04-2024,
row_4,Aquisição de Serviços de Reparação de Telhado ...,Ajuste Direto Regime Geral,Escola Profissional Agrícola Quinta da Lageosa...,"António Ventura, Unipessoal, Lda.","760,00 €",02-04-2024,
row_5,Aquisição de dispositivos de proteção individual,Concurso público,"Centro Hospitalar Universitário Cova da Beira,...","BARNA IMPORT MEDICA PORTUGAL-UNIPESSOAL, LDA","24.459,27 €",28-03-2024,
